In [0]:
#(1)ドライブをマウント
#(2)学習済みモデルのあるディレクトリに移動

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive

/content/drive


In [4]:
cd My Drive

/content/drive/My Drive


In [0]:
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from PIL import Image
from torchvision import models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [0]:
from torchvision import datasets

In [7]:
!pip install pytorch-lightning

     |████████████████████████████████| 204kB 32.2MB/s 
     |████████████████████████████████| 829kB 42.9MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b7bd043d42d4844d8240d287ace283c01ac9eea02115814c6bdd42551bc4bb4d
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_pretrained = True  # 学習済みのパラメータを使用
model = models.densenet161(pretrained=use_pretrained).to(device)

In [0]:
model.classifier = nn.Linear(in_features=2208, out_features=10, bias=True).to(device)

In [277]:
#モデルを読み込む
device = torch.device("cuda")
#model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load("check.pt"))
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [0]:
transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224), #サイズ合わせ
    transforms.RandomHorizontalFlip(), 
    #transforms.RandomVerticalFlip(),
    transforms.ToTensor() ,              #型変換(?)
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  #https://teratail.com/questions/234027
    
    ])

In [0]:
your_datasets = datasets.ImageFolder(root="cholab", transform=transform_test) 

In [0]:
batch_num = 10

In [0]:
loader = torch.utils.data.DataLoader(your_datasets, batch_size= batch_num)

In [282]:
for batch_index, (X, y) in enumerate(loader):
  type(X)
  type(y)

type(X)
type(y)

torch.Tensor

In [0]:
X = X.cuda()

In [284]:
X.shape

torch.Size([10, 3, 224, 224])

In [285]:
y.shape

torch.Size([10])

In [286]:
X

tensor([[[[-0.6281, -0.5596, -0.4226,  ..., -0.9534, -1.0390, -1.0733],
          [-0.5767, -0.5253, -0.4397,  ..., -1.0562, -1.1418, -1.1760],
          [-0.5596, -0.5253, -0.4739,  ..., -1.0904, -1.1589, -1.1932],
          ...,
          [-0.5253, -0.5424, -0.5938,  ..., -0.7650, -0.7479, -0.7308],
          [-0.5253, -0.5424, -0.5767,  ..., -0.7650, -0.7479, -0.7308],
          [-0.5424, -0.5596, -0.5767,  ..., -0.7479, -0.7308, -0.7308]],

         [[-0.7577, -0.6877, -0.5301,  ..., -1.3004, -1.3880, -1.4405],
          [-0.7052, -0.6527, -0.5476,  ..., -1.3880, -1.4755, -1.5105],
          [-0.6702, -0.6352, -0.5651,  ..., -1.3704, -1.4580, -1.4930],
          ...,
          [-0.3550, -0.3725, -0.4251,  ..., -0.6176, -0.6001, -0.5826],
          [-0.3725, -0.3901, -0.4076,  ..., -0.6176, -0.6001, -0.5826],
          [-0.4076, -0.4076, -0.4076,  ..., -0.6001, -0.5826, -0.5826]],

         [[-0.3055, -0.2358, -0.0964,  ..., -0.8633, -0.9156, -0.9504],
          [-0.2707, -0.2184, -

In [287]:
y

tensor([9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [288]:
y[0]

tensor(9)

In [0]:
# 評価モードにする
model = model.eval()

In [290]:
y_predict = model.forward(X)
y_predict

tensor([[-4.0517e-01, -1.3382e+00,  7.3410e+00,  1.9922e-01, -2.6012e+00,
         -3.3525e-01, -2.5610e+00, -9.7820e-01, -1.1593e+00,  2.0741e+00],
        [-1.8994e-01, -1.2860e+00,  4.3090e+00, -8.9186e-01, -3.0403e+00,
          5.4616e-03, -1.3354e+00, -1.2692e+00, -6.7004e-03,  3.7895e+00],
        [-1.2250e+00, -2.0684e+00,  7.7024e+00, -1.0667e+00, -2.1768e+00,
         -3.5381e-01, -1.3640e+00, -1.7497e+00, -1.2599e-01,  3.3013e+00],
        [ 4.5913e-01, -1.7549e+00,  3.9942e+00, -1.3898e+00, -1.4978e+00,
         -7.7062e-01,  9.7480e-02, -9.6142e-01, -8.3875e-01,  3.5020e+00],
        [-1.1233e+00, -1.5426e+00,  8.8969e+00,  1.8951e-01, -3.0484e+00,
         -3.9218e-01, -2.3449e+00, -5.6831e-01, -1.6343e+00,  2.5501e+00],
        [ 1.1722e-01, -1.1866e+00,  1.0697e+00, -4.0883e-01,  2.0129e+00,
         -6.2595e-01,  6.2694e-01,  9.1108e-02, -5.2922e-01,  5.1206e-01],
        [-2.0342e+00, -1.1029e+00,  1.5191e-01, -7.5348e-01, -3.3261e+00,
          1.7260e-01, -5.9511e-0

In [291]:
for i in range(batch_num):
   print(y_predict[i])

tensor([-0.4052, -1.3382,  7.3410,  0.1992, -2.6012, -0.3353, -2.5610, -0.9782,
        -1.1593,  2.0741], device='cuda:0', grad_fn=<SelectBackward>)
tensor([-0.1899, -1.2860,  4.3090, -0.8919, -3.0403,  0.0055, -1.3354, -1.2692,
        -0.0067,  3.7895], device='cuda:0', grad_fn=<SelectBackward>)
tensor([-1.2250, -2.0684,  7.7024, -1.0667, -2.1768, -0.3538, -1.3640, -1.7497,
        -0.1260,  3.3013], device='cuda:0', grad_fn=<SelectBackward>)
tensor([ 0.4591, -1.7549,  3.9942, -1.3898, -1.4978, -0.7706,  0.0975, -0.9614,
        -0.8388,  3.5020], device='cuda:0', grad_fn=<SelectBackward>)
tensor([-1.1233, -1.5426,  8.8969,  0.1895, -3.0484, -0.3922, -2.3449, -0.5683,
        -1.6343,  2.5501], device='cuda:0', grad_fn=<SelectBackward>)
tensor([ 0.1172, -1.1866,  1.0697, -0.4088,  2.0129, -0.6259,  0.6269,  0.0911,
        -0.5292,  0.5121], device='cuda:0', grad_fn=<SelectBackward>)
tensor([-2.0342, -1.1029,  0.1519, -0.7535, -3.3261,  0.1726, -0.5951, -0.9707,
        -0.6674,  9.

In [292]:
for i in range(batch_num):
   print(y_predict[i].argmax())

tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(4, device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')


In [0]:
class_name = ["airplane","bird","car","cat","deer","dog","horse","monkey","ship","tuck"]

In [297]:
for i in range(batch_num):
   print(class_name[y_predict[i].argmax()])

car
car
car
car
car
deer
tuck
tuck
tuck
tuck
